In [ ]:
from pycaret.utils import enable_colab
enable_colab()
import pycaret
from pycaret.regression import *
from pycaret.anomaly import *

from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/날씨')

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid' )
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objects as go
import scikitplot
import sklearn
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import ensemble

Colab mode enabled.


/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('형한결/all.csv')
df = data.copy()

In [ ]:
list_999=list(df.columns[4:21])
list_999

['uv',
 'band1',
 'band2',
 'band3',
 'band4',
 'band5',
 'band6',
 'band7',
 'band8',
 'band9',
 'band10',
 'band11',
 'band12',
 'band13',
 'band14',
 'band15',
 'band16']

In [ ]:
for column in list_999:
  df[column] = df[column].replace(-999,np.nan)

In [ ]:
df.head()

,stn,date_time,lon,lat,uv,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,band11,band12,band13,band14,band15,band16,solarza,sateza,esr,height,landtype
0,13,2020-01-01 00:00:00,126.16000,33.30000,0.0,0.00000,0.00000,0.00000,-0.00075,0.0000,-0.00065,268.21695,241.02138,250.86386,257.12412,267.79360,250.75924,269.97608,269.70811,268.61702,259.10334,165.17963,38.96359,-9.50910,71.00,0
1,105,2020-01-01 00:00:00,128.89098,37.75147,0.0,-0.00057,0.00057,0.00030,-0.00030,0.0003,0.00000,268.21695,240.43948,249.09365,254.80759,267.05264,246.67115,268.22810,268.41686,268.01723,256.99670,163.20549,43.95463,-9.40930,26.04,3
2,108,2020-01-01 00:00:00,126.96500,37.57100,NaN,0.00000,0.00000,-0.00089,-0.00075,-0.0006,0.00000,252.42750,239.67525,248.06191,251.25534,255.94581,241.24994,258.33982,258.18692,257.58363,250.50021,162.48279,43.77999,-9.37761,85.50,3
3,112,2020-01-01 00:00:00,126.62490,37.47772,0.0,-0.00057,-0.00057,-0.00060,0.00015,0.0000,-0.00065,253.87311,239.41633,247.94558,251.20427,255.90395,241.55488,258.54927,258.64012,258.03883,251.00820,162.39995,43.67046,-9.37422,68.99,3
4,115,2020-01-01 00:00:00,130.89864,37.48129,0.0,0.00000,-0.00057,-0.00030,0.00075,0.0006,-0.00065,261.68631,239.67525,247.77042,252.43862,262.29638,244.56708,264.88004,263.95472,262.33534,253.25397,164.19725,43.74521,-9.45200,222.80,4


In [ ]:
df_13=df.set_index('stn').loc[13].set_index('date_time')
df_13.index = pd.to_datetime(df_13.index)

In [ ]:
df_13['year'] = df_13.index.year
df_13['month']=df_13.index.month
df_13['day']=df_13.index.day
df_13['hour']=df_13.index.hour
df_13['quarter']=df_13.index.quarter

In [ ]:
df_13.head()

,lon,lat,uv,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,band11,band12,band13,band14,band15,band16,solarza,sateza,esr,height,landtype,year,month,day,hour,quarter
date_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,126.16,33.3,0.0,0.00000,0.00000,0.00000,-0.00075,0.0000,-0.00065,268.21695,241.02138,250.86386,257.12412,267.79360,250.75924,269.97608,269.70811,268.61702,259.10334,165.17963,38.96359,-9.5091,71.0,0,2020,1,1,0,1
2020-01-01 00:10:00,126.16,33.3,0.0,-0.00057,0.00000,0.00119,-0.00015,-0.0003,-0.00065,264.78634,241.10364,250.78261,256.69732,265.52586,249.26338,267.39961,267.63692,266.66849,258.21621,166.67178,38.96359,-9.5091,71.0,0,2020,1,1,0,1
2020-01-01 00:20:00,126.16,33.3,0.0,0.00000,0.00000,-0.00030,0.00030,0.0000,-0.00065,265.59620,241.18569,250.75549,256.76497,266.14464,249.61859,268.27856,268.19710,267.20654,258.38663,167.94722,38.96359,-9.5091,71.0,0,2020,1,1,0,1
2020-01-01 00:30:00,126.16,33.3,0.0,-0.00057,0.00000,-0.00030,0.00030,-0.0006,-0.00065,266.50471,241.18569,250.67403,256.89999,266.87906,249.79544,268.44652,268.07856,267.50047,258.67009,168.93011,38.96359,-9.5091,71.0,0,2020,1,1,0,1
2020-01-01 00:40:00,126.16,33.3,0.0,-0.00113,-0.00057,-0.00119,-0.00030,-0.0003,-0.00065,265.72831,240.85620,250.37389,256.62956,266.67011,249.86163,268.64762,268.58556,267.34496,258.50010,169.53722,38.96359,-9.5091,71.0,0,2020,1,1,0,1


In [ ]:
for column in list_999:
  df[column] = df[column].replace(-999,np.nan)

In [ ]:
len(df_13)

105264

In [ ]:
len(df_13.loc['2021-07-01 00:00:00':'2021-08-31 23:50:00',:])

8928

In [ ]:
df_13['year'] = df_13.index.year
df_13['month']=df_13.index.month
df_13['day']=df_13.index.day
df_13['hour']=df_13.index.hour
df_13['quarter']=df_13.index.quarter

In [ ]:
df_13.columns

Index(['lon', 'lat', 'uv', 'band1', 'band2', 'band3', 'band4', 'band5',
       'band6', 'band7', 'band8', 'band9', 'band10', 'band11', 'band12',
       'band13', 'band14', 'band15', 'band16', 'solarza', 'sateza', 'esr',
       'height', 'landtype', 'year', 'month', 'day', 'hour', 'quarter'],
      dtype='object')

### imputer='lightgbm', non Stationarity







In [ ]:
from pycaret.regression import *
df_13_setup = pycaret.regression.setup(df_13,
                                   ignore_features=['lon','lat','landtype','sateza','height'],                   
                                   #numeric_features='month',
                                   #date_features='date_time',
                                   train_size=0.7,
                                   target='uv', 
                                   imputation_type='iterative', 
                                   session_id =42,
                                   iterative_imputation_iters=5,
                                   numeric_iterative_imputer='lightgbm')     

,Description,Value
0,session_id,42
1,Target,uv
2,Original Data,"(105264, 29)"
3,Missing Values,True
4,Numeric Features,20
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(65052, 34)"


In [ ]:
collinearity_columns

Index(['band1', 'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9',
       'band10', 'band11', 'band12', 'band15', 'band16', 'solarza', 'esr',
       'day', 'hour', 'year_2021', 'month_1', 'month_10', 'month_11',
       'month_12', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'quarter_1', 'quarter_2', 'quarter_3',
       'quarter_4'],
      dtype='object')

In [ ]:
top5 = pycaret.regression.compare_models(n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.1332,1.136000e-01,0.3370,9.719000e-01,0.0937,0.2341,12.277
lightgbm,Light Gradient Boosting Machine,0.1518,1.283000e-01,0.3581,9.682000e-01,0.1015,0.2807,0.483
rf,Random Forest Regressor,0.1434,1.287000e-01,0.3586,9.681000e-01,0.0990,0.2500,27.170
gbr,Gradient Boosting Regressor,0.1880,1.775000e-01,0.4212,9.560000e-01,0.1152,0.3235,11.386
knn,K Neighbors Regressor,0.1908,2.426000e-01,0.4922,9.400000e-01,0.1343,0.3464,0.874
dt,Decision Tree Regressor,0.2038,2.692000e-01,0.5185,9.334000e-01,0.1409,0.3227,0.446
ada,AdaBoost Regressor,0.5636,4.962000e-01,0.7022,8.773000e-01,0.3468,0.9906,5.382
lr,Linear Regression,0.8597,1.353000e+00,1.1630,6.652000e-01,0.4762,1.9069,0.372
br,Bayesian Ridge,0.8598,1.352900e+00,1.1630,6.652000e-01,0.4764,1.9072,0.123
ridge,Ridge Regression,0.8633,1.353300e+00,1.1632,6.651000e-01,0.4791,1.9122,0.033


In [ ]:
collinearity_columns

Index(['band1', 'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9',
       'band10', 'band11', 'band12', 'band15', 'band16', 'solarza', 'esr',
       'day', 'hour', 'year_2021', 'month_1', 'month_10', 'month_11',
       'month_12', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'quarter_1', 'quarter_2', 'quarter_3',
       'quarter_4'],
      dtype='object')

### imputer='lr', non Stationarity


In [ ]:
from pycaret.regression import *
df_13_setup = pycaret.regression.setup(df_13,
                                   ignore_features=['lon','lat','landtype','sateza','height'],                   
                                   #numeric_features='month',
                                   #date_features='date_time',
                                   train_size=0.7,
                                   target='uv',
                                   imputation_type='iterative', 
                                   session_id =42,
                                   iterative_imputation_iters=5,
                                   numeric_iterative_imputer='lr') 

,Description,Value
0,session_id,42
1,Target,uv
2,Original Data,"(105264, 29)"
3,Missing Values,True
4,Numeric Features,20
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(65052, 33)"


In [ ]:
top5 = pycaret.regression.compare_models(n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.1336,1.148000e-01,0.3387,9.716000e-01,0.0942,0.2349,10.642
lightgbm,Light Gradient Boosting Machine,0.1515,1.285000e-01,0.3585,9.682000e-01,0.1016,0.2804,0.444
rf,Random Forest Regressor,0.1437,1.287000e-01,0.3586,9.681000e-01,0.0994,0.2510,23.064
gbr,Gradient Boosting Regressor,0.1870,1.772000e-01,0.4208,9.561000e-01,0.1151,0.3239,9.802
dt,Decision Tree Regressor,0.1999,2.562000e-01,0.5061,9.366000e-01,0.1395,0.3192,0.398
knn,K Neighbors Regressor,0.1952,2.570000e-01,0.5067,9.364000e-01,0.1372,0.3526,0.588
ada,AdaBoost Regressor,0.5169,4.569000e-01,0.6736,8.869000e-01,0.3112,0.9016,4.482
lr,Linear Regression,0.8970,1.412800e+00,1.1886,6.504000e-01,0.4982,2.0030,0.372
br,Bayesian Ridge,0.8972,1.412800e+00,1.1885,6.504000e-01,0.4984,2.0033,0.084
ridge,Ridge Regression,0.8998,1.413300e+00,1.1888,6.503000e-01,0.5005,2.0077,0.030


In [ ]:
a=5

In [ ]:
df_13.columns

Index(['lon', 'lat', 'uv', 'band1', 'band2', 'band3', 'band4', 'band5',
       'band6', 'band7', 'band8', 'band9', 'band10', 'band11', 'band12',
       'band13', 'band14', 'band15', 'band16', 'solarza', 'sateza', 'esr',
       'height', 'landtype', 'year', 'month', 'day', 'hour', 'quarter'],
      dtype='object')

### imputer='lgb',  Stationarity


In [ ]:
from pycaret.regression import *
df_13_setup = pycaret.regression.setup(df_13,
                                   ignore_features=['lon','lat','landtype','sateza','height','year','month','day','hour','quarter'],                   
                                   #numeric_features='month',
                                   #date_features='date_time',
                                   train_size=0.7,
                                   target='uv',
                                   imputation_type='iterative', 
                                   session_id =42,
                                   iterative_imputation_iters=5,
                                   numeric_iterative_imputer='lightgbm')    

,Description,Value
0,session_id,42
1,Target,uv
2,Original Data,"(105264, 29)"
3,Missing Values,True
4,Numeric Features,18
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(65052, 14)"


In [ ]:
top5 = pycaret.regression.compare_models(n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.1632,0.1627,0.4032,0.9597,0.1098,0.2783,5.832
rf,Random Forest Regressor,0.1664,0.1672,0.4088,0.9586,0.1108,0.2810,18.764
lightgbm,Light Gradient Boosting Machine,0.1772,0.1717,0.4142,0.9575,0.1137,0.3136,0.353
gbr,Gradient Boosting Regressor,0.2064,0.2177,0.4664,0.9461,0.1242,0.3527,8.475
knn,K Neighbors Regressor,0.2163,0.2997,0.5471,0.9258,0.1482,0.3853,0.261
dt,Decision Tree Regressor,0.2331,0.3447,0.5869,0.9147,0.1546,0.3530,0.320
ada,AdaBoost Regressor,0.4289,0.4382,0.6580,0.8912,0.2478,0.7627,2.861
lr,Linear Regression,0.9013,1.4285,1.1951,0.6465,0.4991,2.0271,0.407
br,Bayesian Ridge,0.9013,1.4285,1.1951,0.6465,0.4991,2.0272,0.040
ridge,Ridge Regression,0.9038,1.4291,1.1954,0.6464,0.5011,2.0308,0.030


### imputer='lgb' 


In [ ]:
from pycaret.regression import *
df_13_setup_lightgbm = pycaret.regression.setup(df_13,
                                   session_id =42,
                                   target='uv', 
                                   ignore_features=['lon','lat','landtype','sateza','height'],                   
                                   #numeric_features=[],
                                   #date_features=[],
                                   #categorical_features=[],
                                   train_size=0.9999,
                                   imputation_type='iterative', 
                                   iterative_imputation_iters=5,
                                   numeric_iterative_imputer='lightgbm',
                                   categorical_iterative_imputer="lightgbm",
                                   remove_perfect_collinearity=True) 

,Description,Value
0,session_id,42
1,Target,uv
2,Original Data,"(105264, 29)"
3,Missing Values,True
4,Numeric Features,20
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(92943, 34)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True,
                                      features_todrop=['lon', 'lat', 'landtype',
                                                       'sateza', 'height'],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='uv',
                                      time_features=[])),
                ('imputer',
                 Iterative_Imputer(add_indicator=False,
                                   classifier=LGBMClassifier(boosting_type='gbdt',
                                                             cl...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'),

In [ ]:
lgb_top5 = pycaret.regression.compare_models(n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.1272,1.050000e-01,0.3240,9.738000e-01,0.0907,0.2250,16.270
rf,Random Forest Regressor,0.1386,1.212000e-01,0.3480,9.698000e-01,0.0967,0.2428,36.195
lightgbm,Light Gradient Boosting Machine,0.1505,1.251000e-01,0.3536,9.688000e-01,0.1009,0.2784,0.588
gbr,Gradient Boosting Regressor,0.1879,1.769000e-01,0.4205,9.559000e-01,0.1155,0.3257,15.001
knn,K Neighbors Regressor,0.1713,2.071000e-01,0.4549,9.484000e-01,0.1252,0.3133,0.873
dt,Decision Tree Regressor,0.1944,2.476000e-01,0.4975,9.383000e-01,0.1369,0.3088,0.600
ada,AdaBoost Regressor,0.5704,5.054000e-01,0.7098,8.741000e-01,0.3491,1.0120,7.377
lr,Linear Regression,0.8598,1.346800e+00,1.1605,6.644000e-01,0.4776,1.9073,0.424
br,Bayesian Ridge,0.8599,1.346800e+00,1.1605,6.644000e-01,0.4777,1.9075,0.132
ridge,Ridge Regression,0.8620,1.347100e+00,1.1606,6.643000e-01,0.4793,1.9110,0.034


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:[ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=42, verbose=0, warm_start=False), RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs

## imputer = lr

In [ ]:
from pycaret.regression import *
df_13_setup_lr = pycaret.regression.setup(df_13,
                                   session_id =42,
                                   target='uv', 
                                   ignore_features=['lon','lat','landtype','sateza','height'],                   
                                   #numeric_features=[],
                                   #date_features=[],
                                   #categorical_features=[],
                                   train_size=0.9999,
                                   imputation_type='iterative', 
                                   iterative_imputation_iters=5,
                                   numeric_iterative_imputer='lr',
                                   categorical_iterative_imputer="lr",
                                   remove_perfect_collinearity=True) 

,Description,Value
0,session_id,42
1,Target,uv
2,Original Data,"(105264, 29)"
3,Missing Values,True
4,Numeric Features,20
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(92943, 34)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True,
                                      features_todrop=['lon', 'lat', 'landtype',
                                                       'sateza', 'height'],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='uv',
                                      time_features=[])),
                ('imputer',
                 Iterative_Imputer(add_indicator=False,
                                   classifier=LogisticRegression(C=1.0,
                                                                 class_weight=...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthroug

In [ ]:
top5 = pycaret.regression.compare_models(n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.1274,1.055000e-01,0.3247,9.737000e-01,0.0908,0.2255,15.542
rf,Random Forest Regressor,0.1385,1.208000e-01,0.3476,9.699000e-01,0.0969,0.2438,36.125
lightgbm,Light Gradient Boosting Machine,0.1508,1.260000e-01,0.3549,9.686000e-01,0.1013,0.2820,0.588
gbr,Gradient Boosting Regressor,0.1874,1.775000e-01,0.4212,9.558000e-01,0.1155,0.3248,14.640
knn,K Neighbors Regressor,0.1704,2.046000e-01,0.4521,9.490000e-01,0.1243,0.3114,0.935
dt,Decision Tree Regressor,0.1936,2.453000e-01,0.4951,9.389000e-01,0.1364,0.3100,0.610
ada,AdaBoost Regressor,0.6072,5.510000e-01,0.7379,8.626000e-01,0.3742,1.0829,6.983
lr,Linear Regression,0.8837,1.389800e+00,1.1789,6.537000e-01,0.4903,1.9633,0.465
br,Bayesian Ridge,0.8838,1.389800e+00,1.1789,6.537000e-01,0.4904,1.9635,0.169
ridge,Ridge Regression,0.8860,1.390100e+00,1.1790,6.536000e-01,0.4921,1.9669,0.040


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:[ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=42, verbose=0, warm_start=False), RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs

In [ ]:
df_146 = df.set_index('stn').loc[146]

In [ ]:
from pycaret.regression import *
df_13_setup_knn = pycaret.regression.setup(df_13,
                                   session_id =42,
                                   target='uv', 
                                   ignore_features=['lon','lat','landtype','sateza','height'],                   
                                   #numeric_features=[],
                                   #date_features=[],
                                   #categorical_features=[],
                                   train_size=0.9999,
                                   imputation_type='iterative', 
                                   iterative_imputation_iters=5,
                                   numeric_iterative_imputer='knn',
                                   categorical_iterative_imputer="knn",
                                   remove_perfect_collinearity=True) 

,Description,Value
0,session_id,42
1,Target,uv
2,Original Data,"(105264, 29)"
3,Missing Values,True
4,Numeric Features,20
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(92943, 34)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True,
                                      features_todrop=['lon', 'lat', 'landtype',
                                                       'sateza', 'height'],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='uv',
                                      time_features=[])),
                ('imputer',
                 Iterative_Imputer(add_indicator=False,
                                   classifier=KNeighborsClassifier(algorithm='auto',...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_al

In [ ]:
pycaret.regression.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.1280,1.064000e-01,0.3261,9.735000e-01,0.0913,0.2263,17.881
rf,Random Forest Regressor,0.1394,1.228000e-01,0.3503,9.694000e-01,0.0973,0.2444,40.956
lightgbm,Light Gradient Boosting Machine,0.1514,1.263000e-01,0.3553,9.685000e-01,0.1014,0.2800,0.628
gbr,Gradient Boosting Regressor,0.1882,1.777000e-01,0.4215,9.557000e-01,0.1157,0.3270,16.480
knn,K Neighbors Regressor,0.1730,2.104000e-01,0.4584,9.476000e-01,0.1265,0.3166,1.329
dt,Decision Tree Regressor,0.1959,2.515000e-01,0.5013,9.373000e-01,0.1373,0.3135,0.682
ada,AdaBoost Regressor,0.5809,5.363000e-01,0.7248,8.664000e-01,0.3550,1.0438,8.446
lr,Linear Regression,0.8553,1.345700e+00,1.1600,6.647000e-01,0.4739,1.9028,0.408
br,Bayesian Ridge,0.8554,1.345700e+00,1.1600,6.647000e-01,0.4740,1.9031,0.176
ridge,Ridge Regression,0.8580,1.345900e+00,1.1601,6.646000e-01,0.4761,1.9070,0.036


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=42, verbose=0, warm_start=False)
INFO:logs:compare_models() succesfully completed......................................


ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=42, verbose=0, warm_start=False)